In [ ]:
import sys
import time
import asyncio
import json
from datetime import datetime
from openai import OpenAI

client = OpenAI(api_key="sk-proj-xxxxxxxxxxxxxx",
)

# Global variable to store the EFD client
efd_client = None

def init_efd_client():
    """
    Initialize the EFD client.
    """
    global efd_client
    import nest_asyncio
    nest_asyncio.apply()
    from lsst_efd_client import EfdClient
    from lsst.sitcom.vandv.efd import create_efd_client
    efd_client = create_efd_client()
    return "EFD client initialized successfully."

async def list_topics():
    """
    List all available topics in the EFD.
    """
    global efd_client
    if efd_client is None:
        return "Error: EFD client not initialized. Please call init_efd_client first."
    topics = await efd_client.get_topics()
    return topics

async def list_fields(topic):
    """
    List all fields available for a given topic.
    """
    global efd_client
    if efd_client is None:
        return "Error: EFD client not initialized. Please call init_efd_client first."
    fields = await efd_client.get_fields(topic)
    return fields

async def get_efd_data(topic, fields, start_time, end_time):
    """
    Retrieve data from the EFD for a given topic, fields, and time range.

    Parameters:
    - topic (str): The EFD topic to query.
    - fields (list): List of fields to retrieve.
    - start_time (str): Start time in ISO format (e.g., "2023-03-10 03:00:00Z").
    - end_time (str): End time in ISO format (e.g., "2023-03-10 03:30:00Z").
    """
    global efd_client
    if efd_client is None:
        return "Error: EFD client not initialized. Please call init_efd_client first."
    from astropy.time import Time

    try:
        start = Time(start_time, scale='utc')
        end = Time(end_time, scale='utc')
    except ValueError as e:
        return f"Error parsing time: {e}"

    try:
        data = await efd_client.select_time_series(topic, fields, start.utc, end.utc)
        return data.to_json()
    except Exception as e:
        return f"Error retrieving data: {e}"

async def handle_assistant_response(tool_call):
    """
    Handle the assistant's tool calls by executing the corresponding functions.
    """
    function_name = tool_call["function_name"]
    arguments = tool_call.get("arguments", {})

    if function_name == 'init_efd_client':
        return init_efd_client()
    elif function_name == 'list_topics':
        topics = await list_topics()
        return json.dumps(topics)
    elif function_name == 'list_fields':
        topic = arguments.get('topic')
        if not topic:
            return "Error: 'topic' argument is required."
        fields = await list_fields(topic)
        return json.dumps(fields)
    elif function_name == 'get_efd_data':
        topic = arguments.get('topic')
        fields = arguments.get('fields')
        start_time = arguments.get('start_time')
        end_time = arguments.get('end_time')
        if not all([topic, fields, start_time, end_time]):
            return "Error: All arguments ('topic', 'fields', 'start_time', 'end_time') are required."
        data = await get_efd_data(topic, fields, start_time, end_time)
        return data
    else:
        return f"Unknown function: {function_name}"

async def ask_assistant(question):
    """
    Send a question to the assistant and handle the response.
    """
    # Implement the interaction with OpenAI's assistant here.
    # This is a placeholder for demonstration purposes.
    # In practice, you would send the question to the assistant and process its response.
    pass

# Update the assistant creation to reflect these changes
my_assistant = client.beta.assistants.create(
    model="gpt-4o-mini",
    name="EFD Assistant",
    description="A personal EFD database assistant that can search, answer questions, and generate plots.",
    instructions="""You are a personal assistant for the EFD database. When asked a question, follow these steps:

1. Initialize the EFD client using `init_efd_client()`.
2. Use `list_topics()` to retrieve available topics if needed.
3. Use `list_fields(topic)` to retrieve available fields for a topic if needed.
4. Use `get_efd_data(topic, fields, start_time, end_time)` to retrieve data.
5. Ensure that `start_time` and `end_time` are in ISO format (e.g., "2023-03-10 03:00:00Z").
6. Provide clear and informative responses based on the data retrieved.
""",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "init_efd_client",
                "description": "Initialize the EFD client.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "list_topics",
                "description": "List all available topics in the EFD.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                    "name": "list_fields",
                    "description": "List all fields for a given topic.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "topic": {"type": "string", "description": "The EFD topic to query."}
                        },
                        "required": ["topic"]
                    }
                }
        },
        {
            "type": "function",
            "function": {
                "name": "get_efd_data",
                "description": "Retrieve data from the EFD for a given topic, fields, and time range.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "topic": {"type": "string", "description": "The EFD topic to query."},
                        "fields": {"type": "array", "items": {"type": "string"}, "description": "List of fields to retrieve."},
                        "start_time": {"type": "string", "description": "Start time in ISO format."},
                        "end_time": {"type": "string", "description": "End time in ISO format."}
                    },
                    "required": ["topic", "fields", "start_time", "end_time"]
                }
            }
        }
    ],
    temperature=0.1,
    top_p=0.1
)

print(my_assistant)

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

# Initialize the EFD client
response = await handle_assistant_response({
    "function_name": "init_efd_client",
    "arguments": {}
})
print(response)

# List all topics
response = await handle_assistant_response({
    "function_name": "list_topics",
    "arguments": {}
})
topics = json.loads(response)
print(topics)


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
import json


In [ ]:
response = await handle_assistant_response({
    "function_name": "list_fields",
    "arguments": {"topic": "lsst.sal.MTMount.azimuth"}
})
fields = json.loads(response)
print(fields)


In [ ]:
response = await handle_assistant_response({
    "function_name": "get_efd_data",
    "arguments": {
        "topic": "lsst.sal.MTMount.azimuth",
        "fields": ["actualPosition", "timestamp"],
        "start_time": "2023-03-10 03:00:00Z",
        "end_time": "2023-03-10 03:30:00Z"
    }
})
import pandas as pd
data = pd.read_json(response)
print(data)
